<a href="https://colab.research.google.com/github/danhively/CS4499-Data-Science-and-Applied-Machine-Learning/blob/main/Mid_Project_Hively.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mid-Project: Predicting Sports Winners
Data Science and Applied Machine Learning \\
Adpopted from material from Dr. Kerby \\
Student Name: Dan Hively \\
Date Due: April 5, 2023, 11:59 PM

### Question 1: Gather Data from 2021-2022
*Gather* data from [Basketball Reference](https://www.basketball-reference.com/leagues/NBA_2022.html) <br>

Data can be downloaded by clicking on the `Share and Export` drop-down list at the middle top of available tables. Start off by clicking `Schedule and Results` and downloading the monthly games tables. Combine these into one table. Then import the full games-results table into python. 


In [19]:
# Imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# Gather data - This will be the baseline data.
# 2021-22 Season.
csv_202122 = 'https://raw.github.com/danhively/CS4499/main/Mid-Poject/NBA_2021-22.csv'

# 202-21 Season.
csv_202021 = 'https://raw.github.com/danhively/CS4499/main/Mid-Poject/NBA_2020-21.csv'

#######################################
#   Useful Functions                  #
#######################################

# Function to rename a column by position
def rename_column_by_position(df, position, new_name):
    if 0 <= position < len(df.columns):
        columns = df.columns.tolist()
        columns[position] = new_name
        df.columns = columns
    else:
        raise ValueError(f"Invalid position. DataFrame has {len(df.columns)} columns.")

# Function to compare PTS1 and PTS
def compare_scores(row):
    if row['PTS.1'] > row['PTS']:
        return True
    else:
        return False

# Drop specified columns from a DataFrame.
def drop_columns(df, columns_to_drop):
    return df.drop(columns=columns_to_drop)        

# Reads a CSV file from the given URL and returns a pandas DataFrame.
def read_csv_to_dataframe(csv_url):
    return pd.read_csv(csv_url)    


dfNBA202122 = read_csv_to_dataframe(csv_202122)

# This will be commented out before submission.
# Mount my Google Drive
# I want to save a CSV after manipulations to check it.
#from google.colab import drive
#mntPath = '/content/drive/'
#drive.mount(mntPath, force_remount=True)
#svPath = mntPath + "My Drive/CS4499/Mid-Project/"
#dfNBA202122.to_csv(svPath + "dfNBA202122.csv")



### Question 2: EDA and Preprocessing
Explore your dataset and make any preprocessing adjustments necessary. You must perform at least 3 operations on exploring your data. Each one must provide unqiue insight to the data. At the bottom of this problem, write a paragraph (at least 5 sentences) explaining what you did and found out. 

In [20]:
# Print some basic information about the dataset.
# Print the shape of the DataFrame
print('----------------------------')
print('Shape')
print('----------------------------')
print(dfNBA202122.shape)

# Print some summary statistics of the DataFrame
print('----------------------------')
print('statistics')
print('----------------------------')
print(dfNBA202122.describe())

# Print the list of columns in the CSV file.
print('----------------------------')
print('Columns')
print('----------------------------')
print(dfNBA202122.columns)

# Print the first five rows of the DataFrame
print('----------------------------')
print('First 5 rows')
print('----------------------------')
print(dfNBA202122.head())

# Count the number of NaN values in each column
print('----------------------------')
print('Number of NaN values')
print('----------------------------')
nan_count = dfNBA202122.isna().sum()
print(nan_count)

# Find columns with NaN values
null_columns = dfNBA202122.columns[dfNBA202122.isna().any()].tolist()

# Display column headings with null values
print('----------------------------')
print('Column headings with null values')
print('----------------------------')
print(null_columns)

----------------------------
Shape
----------------------------
(1171, 11)
----------------------------
statistics
----------------------------
               PTS        PTS.1  Notes
count  1171.000000  1171.000000    0.0
mean    111.382579   112.558497    NaN
std      12.563364    12.510739    NaN
min      75.000000    73.000000    NaN
25%     103.000000   104.000000    NaN
50%     111.000000   112.000000    NaN
75%     120.000000   121.000000    NaN
max     154.000000   154.000000    NaN
----------------------------
Columns
----------------------------
Index(['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1',
       'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes'],
      dtype='object')
----------------------------
First 5 rows
----------------------------
                Date Start (ET)        Visitor/Neutral  PTS  \
0  Tue, Dec 22, 2020      7:00p  Golden State Warriors   99   
1  Tue, Dec 22, 2020     10:00p   Los Angeles Clippers  116   
2  Wed, Dec 

What I discoverwed about the data.


The data

### Question 3: Feature Engineering

Create a column for whether or not the home team won the game; call it `HomeWin`. Use Pandas and your knowledge of Python to fill in this data. This will become our target -- ie what we are trying to predict. 

In [21]:
# Create a copy of the dfNBA202122 DataFrame
winsdf = dfNBA202122.copy()

# We have 2 columns that have the same name. Let's rename them.
# Rename the columns
#rename_column_by_position(winsdf, 3, 'V_Score')
#rename_column_by_position(winsdf, 5, 'H_Score')

# Create a new column 'HomeWin' based on whether 'PTS.1' is greater than 'PTS' .
# True if PTS.1 is greater than PTS otherwise False.
winsdf['HomeWin'] = winsdf.apply(compare_scores, axis=1)

# Display the new DataFrame with the added column.
print("\nNew DataFrame with 'HomeWin' column (winsdf):")
print(winsdf.columns.tolist())
print(winsdf.head())
print(winsdf['HomeWin'])


New DataFrame with 'HomeWin' column (winsdf):
['Date', 'Start (ET)', 'Visitor/Neutral', 'PTS', 'Home/Neutral', 'PTS.1', 'Unnamed: 6', 'Unnamed: 7', 'Attend.', 'Arena', 'Notes', 'HomeWin']
                Date Start (ET)        Visitor/Neutral  PTS  \
0  Tue, Dec 22, 2020      7:00p  Golden State Warriors   99   
1  Tue, Dec 22, 2020     10:00p   Los Angeles Clippers  116   
2  Wed, Dec 23, 2020      7:00p      Charlotte Hornets  114   
3  Wed, Dec 23, 2020      7:00p        New York Knicks  107   
4  Wed, Dec 23, 2020      7:00p             Miami Heat  107   

          Home/Neutral  PTS.1 Unnamed: 6 Unnamed: 7 Attend.  \
0        Brooklyn Nets    125  Box Score        NaN       0   
1   Los Angeles Lakers    109  Box Score        NaN       0   
2  Cleveland Cavaliers    121  Box Score        NaN     300   
3       Indiana Pacers    121  Box Score        NaN       0   
4        Orlando Magic    113  Box Score        NaN   3,396   

                        Arena  Notes  HomeWin  
0    

### Question 4: Feature Engineering II
Create two columns for how many games the home and visitor teams have won thus far in the season. Call them `HomeNumWins` and `VisitorNumWins`.
 
For example Game 1 for the Utah Jazz was against the OKC Thunder. The Jazz were the home team, OKC was the road team. Since this is game 1 for both teams `HomeNumWins` and `VisitorNumWins` will be 0. The Jazz won this game and were the road team for their next game (against the Sacramento Kings),  `VisitorNumWins` should be a 1 for this value. OKC would go on to lose their second game (on the road to Houston), Game 3 was their first home game. So for OKC's 3rd game (against the 76ers) `HomeNumWins` would be 0. 

Hint: Store each team and the number of intial wins (0) in a Python dictionary. You can then increment this value as you go through the dataset and use this information to update your feature. 



In [ ]:
# Create two new columns 'HomeNumWins' and 'VisitorNumWins'
winsdf['HomeNumWins'] = 0
winsdf['VisitorNumWins'] = 0

# Dictionary to store wins for each team
team_wins = {}

# Get all team names and print them.
teams = set(winsdf['Home/Neutral']).union(winsdf['Visitor/Neutral'])
print(teams)


# Initialize the dictionary with 0 wins for each team
for team in teams:
    team_wins[team] = 0

for index, row in winsdf.iterrows():
    home_team = row['Home/Neutral']
    visitor_team = row['Visitor/Neutral']
    home_win = row['HomeWin']
    
    if home_win:
        team_wins[home_team] += 1
        winsdf.at[index, 'HomeNumWins'] = team_wins[home_team]
        winsdf.at[index, 'VisitorNumWins'] = team_wins[visitor_team]
    else:
        team_wins[visitor_team] += 1
        winsdf.at[index, 'HomeNumWins'] = team_wins[home_team]
        winsdf.at[index, 'VisitorNumWins'] = team_wins[visitor_team]

print(winsdf.head())
print(winsdf.columns.tolist())

# Save the resulting DataFrame to a CSV file just to check it has been
# modified properly. This will be commented out before submission.
#print('===========================================')
#winsdf.to_csv(svPath + "winsdf.csv")
#print("CSV file saved as 'winsdf.csv'")

In [ ]:
# Example with old, 2018-2019, season data
import pandas as pd
import numpy as np

sports = pd.read_csv("https://gitlab.com/CEADS/DrKerby/python/-/raw/master/basketball.csv")
sports

In [24]:
sports['Unnamed: 6'].unique()

array(['Box Score'], dtype=object)

In [25]:
sports['Unnamed: 7'].unique()

array([nan, 'OT', '2OT', '3OT', '4OT'], dtype=object)

In [26]:
sports['Notes'].unique()

array([nan, 'at Mexico City Mexico', 'at London England'], dtype=object)

In [ ]:
from collections import defaultdict
won_last = defaultdict(int)

won_last
# Create HomeWin column
sports['HomeWin'] = sports['PTS.1'] > sports['PTS']
sports.head()
# Create new columns and initialize with zero
sports['HomeLastWin'] = 0
sports['VisitorLastWin'] = 0

sports.head()

# Take a slice first, sports[:5]
for index, row in sports.iterrows():
  #print(index, row['Home/Neutral'], row['PTS.1'])
  home_team = row['Home/Neutral']
  visitor_team = row['Visitor/Neutral']
  sports.loc[index,'HomeLastWin'] = won_last[home_team]
  sports.loc[index,'VisitorLastWin'] = won_last[visitor_team]

  # Who won this game, to record it in the next game?
  #print(int(row['HomeWin']))  # typecast boolean to int (false->0, true->1)
  won_last[home_team] = int(row['HomeWin'])
  won_last[visitor_team] = 1 - int(row['HomeWin'])

sports.head(50)
#type(sports['Date'][0])

### Question 5: Model

Train a decision tree model to determine if the home team won (a 1 is a win and a 0 is a loss. Evaluate your model. At the bottom of this problem write 2 paragraphs explaining your models, its performance and any possible issues you see with it. Comment on the most "important" features in the model.

Remember to split the dataset into training and testing sets and delete any columns that you deem unuseful. (Hint: Do not use any information that you wouldn't know before the game is played, the idea is to create a model that predicts wins)



In [ ]:
# Remove some columns

# Duplicate the dataframe with only columns we need for the model.
new22 = dfNBA202122[['Visitor/Neutral', 'Home/Neutral', 'PTS', 'PTS.1']].copy()

# Create 'HomeWin' column
new22.loc[:, 'HomeWin'] = (new22['PTS'] < new22['PTS.1']).astype(int)
#print(new22)

# Prepare the data for training and testing.
X = new22.drop(columns='HomeWin')
X = pd.get_dummies(X)  
y = new22['HomeWin']

#print(X)
#print(y)

# Split the data into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.7, random_state=35)

# Train the decision tree model
clf = DecisionTreeClassifier(random_state=42)
clf.fit(Xtrain, ytrain)

# Predict the outcomes for the test set
y_pred = clf.predict(Xtest)

importances = clf.feature_importances_
feature_importances = pd.Series(importances, index=X.columns)
top_features = feature_importances.nlargest(5)
print('-----------------------------------------')
print("Top features")
print(top_features)
print('-----------------------------------------')
# Evaluate the model
print("Accuracy:", accuracy_score(ytest, y_pred))
print("Confusion Matrix:\n", confusion_matrix(ytest, y_pred))
print("Classification Report:\n", classification_report(ytest, y_pred))

### Question 6: Add data from the previous season
Go back to the website and download the team standings from the PREVIOUS season. You will find it in the `Standings` tab. Keep just the number of wins and create two new columns called `HomeWins2020` and `VisitorWins2020`.

In [ ]:
# Load the data for the 2021-22 NBA season
#dfNBA202122 = read_csv_to_dataframe(csv_202122)

# Load the data for the 2020-21 NBA season
dfNBA202021 = read_csv_to_dataframe(csv_202021)

# Duplicate the dataframe with only columns we need for the model.
new20 = dfNBA202021[['Visitor/Neutral', 'Home/Neutral', 'PTS', 'PTS.1']].copy()

# Create 'HomeWin' column
new20.loc[:, 'HomeWin'] = (new20['PTS'] < new20['PTS.1']).astype(int)
#print(new20)

# Duplicate the dataframe with only columns we need for the model.
#new22 = dfNBA202122[['Visitor/Neutral', 'Home/Neutral', 'PTS', 'PTS.1']].copy()
# Create 'HomeWin' column
#new22.loc[:, 'HomeWin'] = (new22['PTS'] < new22['PTS.1']).astype(int)
#print(new22)

# Combine the data from both seasons
#fulldf = pd.concat([new22, new20], ignore_index=True)
#print(fulldf)

# Calculate home wins and visitor wins for each team in season 2020-21.
home_wins = new20.groupby('Home/Neutral')['HomeWin'].sum()
visitor_wins = new20.groupby('Visitor/Neutral')['HomeWin'].apply(lambda x: x.shape[0] - x.sum())

# Combine home wins and visitor wins into a single DataFrame
wins_2020 = pd.DataFrame({'HomeWins2020': home_wins, 'VisitorWins2020': visitor_wins}).reset_index()
print(wins_2020)

### Question 7: Repeat Model

Repeat your analysis from Question 5.

In [45]:
# Prepare the data for training and testing.
X = new20.drop(columns='HomeWin')
X = pd.get_dummies(X)  
y = new20['HomeWin']

#print(X)
#print(y)

# Split the data into training and testing sets
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.7, random_state=35)

# Train the decision tree model
clf = DecisionTreeClassifier(random_state=42)
clf.fit(Xtrain, ytrain)

# Predict the outcomes for the test set
y_pred = clf.predict(Xtest)

importances = clf.feature_importances_
feature_importances = pd.Series(importances, index=X.columns)
top_features = feature_importances.nlargest(5)
print('-----------------------------------------')
print("Top features")
print(top_features)
print('-----------------------------------------')
# Evaluate the model
print("Accuracy:", accuracy_score(ytest, y_pred))
print("Confusion Matrix:\n", confusion_matrix(ytest, y_pred))
print("Classification Report:\n", classification_report(ytest, y_pred))

-----------------------------------------
Top features
PTS.1                                  0.474876
PTS                                    0.422808
Home/Neutral_Portland Trail Blazers    0.018745
Visitor/Neutral_Boston Celtics         0.013062
Home/Neutral_Minnesota Timberwolves    0.011026
dtype: float64
-----------------------------------------
Accuracy: 0.9219512195121952
Confusion Matrix:
 [[333  38]
 [ 26 423]]
Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.90      0.91       371
           1       0.92      0.94      0.93       449

    accuracy                           0.92       820
   macro avg       0.92      0.92      0.92       820
weighted avg       0.92      0.92      0.92       820



### Question 8: Analysis and Discussion
What was your best-performing model? \\
How well did it do? \\
Was it overfit? \\
What seems to be the most important feature(s)? \\
How well did your model perform compared to blindly guessing the Home Team won?